<a href="https://colab.research.google.com/github/cornguo/colab/blob/main/CDial_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

資料來源
* https://github.com/thu-coai/CDial-GPT.git

安裝套件

In [ ]:
!git clone https://github.com/thu-coai/CDial-GPT.git
%cd /content/CDial-GPT
!pip install -r requirements.txt

下載模型

In [ ]:
%cd /content/CDial-GPT/data
#!wget https://cloud.tsinghua.edu.cn/f/372be4a9994b4124810e/?dl=1 -O STC-corpus.zip  # 下载 STC 数据集并将其解压至 "data_path" 目录 (如果微调所使用的数据集为 STC)
#!unzip STC-corpus.zip
#!wget https://cloud.tsinghua.edu.cn/f/4dfb8c6c22ae47fbbe98/?dl=1 -O GPT_LCCC-large.zip  # 下载 GPT_LCCC-large 模型权重文件，并将其解压至 "model_checkpoint" 目录
#!unzip GPT_LCCC-large.zip
!wget https://cloud.tsinghua.edu.cn/f/babe2be4f3f747abb887/?dl=1 -O GPT2_LCCC-base.zip
!unzip GPT2_LCCC-base.zip

#!mv ./CDial-GPT2_LCCC-base/vocab.txt ./CDial-GPT2_LCCC-base/vocab.cn.txt
#!apt install cconv
#!cconv -f UTF8-CN -t UTF8-TW ./CDial-GPT2_LCCC-base/vocab.cn.txt -o ./CDial-GPT2_LCCC-base/vocab.txt

Fine-tune

In [ ]:
%cd /content/CDial-GPT
!wget https://raw.githubusercontent.com/zake7749/Gossiping-Chinese-Corpus/master/data/Gossiping-QA-Dataset.txt -O ./data/Gossiping-QA-Dataset.txt

#!PYTHONPATH=$(pwd) python -m torch.distributed.launch --nproc_per_node=2 train.py --pretrained --model_checkpoint ./data/CDial-GPT2_LCCC-base/ --train_path ./data/Gossiping-QA-Dataset.txt --scheduler linear --gpt2 --n_epochs 1 # 以分布式的方式在8块GPU上训练

# 2000
!shuf -n 2000 ./data/Gossiping-QA-Dataset.txt | awk -F '\t' '{print $1"\n"$2}' | sed 's/\(.\)/\1 /g' | sed 's/ $//g' > ./data/Gossiping-QA-Dataset.2000.txt
!PYTHONPATH=$(pwd) python train.py --pretrained --model_checkpoint ./data/CDial-GPT2_LCCC-base/ --train_path ./data/Gossiping-QA-Dataset.2000.txt --scheduler linear --gpt2 --n_epochs 30 # 使用单个GPU进行训练
!mkdir -p "/content/drive/My Drive/tmp/CDial-GPT2_LCCC-base/2000"
!cp -rf /content/CDial-GPT/data/CDial-GPT2_LCCC-base/* "/content/drive/My Drive/tmp/CDial-GPT2_LCCC-base/2000"

測試模型

In [ ]:
%cd /content/CDial-GPT
#!PYTHONPATH=$(pwd) python infer.py --model_checkpoint ./data/CDial-GPT2_LCCC-base/ --datapath STC_test.json --out_path STC_result.txt --gpt2 # 在测试数据上生成回复
!PYTHONPATH=$(pwd) python interact.py --model_checkpoint ./data/CDial-GPT2_LCCC-base/ --gpt2 # 在命令行中与模型进行交互